In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

In [18]:
StocksDF = pd.read_csv('TESLASTOCKS.csv')
StocksDF['Date'] = pd.to_datetime(StocksDF['Date'], dayfirst=True)

In [19]:
Percent_Change = [StocksDF["Percent Change"][i][:-1] for i in range(len(StocksDF["Percent Change"]))]
StocksDF = StocksDF.drop(columns = "Percent Change")
StocksDF["Percent Change"] = Percent_Change

In [20]:
TweetsDF = pd.read_csv('TWEETS.csv')
TweetsDF = TweetsDF.rename(columns = {'Timestamp':'Date'})
TweetsDF = TweetsDF.drop(['Emojis','Retweets','UserScreenName', 'UserName','Comments','Likes','Image link'],axis=1)
TweetsDF['Date'] = pd.to_datetime(TweetsDF['Date'], dayfirst=True)

In [21]:
df = pd.merge(StocksDF, TweetsDF, on = "Date")
df = df.drop_duplicates(subset='Date', keep = "first")
df.reset_index(drop=True, inplace=True)

In [26]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Price Change,Percent Change,Tweet,text,Tweet URL
0,2018-01-03,64.199997,65.050003,63.110001,63.450001,63.450001,22607500,0.628006,1,2992,Come work at the biggest & most advanced facto...,https://twitter.com/elonmusk/status/9484282744...
1,2018-01-05,63.324001,63.448002,62.400002,63.316002,63.316002,22956000,-0.261997,0,2994,"Falcon Heavy goes vertical\n0:19\n2M views\n2,...",https://twitter.com/elonmusk/status/9490754750...
2,2018-01-08,63.200001,67.403999,63.099998,67.281998,67.281998,49297000,3.955997,6,2996,"Long exposure of rocket ascent, reentry from s...",https://twitter.com/elonmusk/status/9504162449...
3,2018-01-09,67.031998,67.760002,65.480003,66.737999,66.737999,35733000,0.356003,1,2998,Replying to \n@PPathole\n @Tesla\n and 2 other...,https://twitter.com/elonmusk/status/9506003636...
4,2018-01-12,67.725998,68.082001,66.734001,67.244003,67.244003,24125500,-0.879997,-1,3006,Replying to \n@ID_AA_Carmack\n and \n@Danielle...,https://twitter.com/elonmusk/status/9518938946...
